##Topic: Mutual Fund Predictive Analytics:

**Members:** 

1) **Ashwini Patil** (ashwini.acpce@gmail.com)

2) **Rupesh More** (rupeshmore85@gmail.com)

**Github Repository:** 

[CS109_Fund_Analytics](https://github.com/rupeshmore85/cs109_Fund_Analytics)


# Web Scraping for fetching the Fund Information:

  Here running the below python code for scraping the data from [www.morningstar.com](http://www.morningstar.com) takes very long time (~ 2 hour 30 mins) for approximately 1277 funds of 10 fund families. Hence the scraped data is exported in csv files and further data analysis in another python file is performed by reading the csv files for faster execution.

### 1. Import Python Libraries

In [1507]:
# Import Section
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 
import requests 
import numpy as np
import pandas as pd                               # pandas
from StringIO import StringIO
import matplotlib.pyplot as plt                   # module for plotting 
import datetime as dt                             # module for manipulating dates and times
from collections import OrderedDict

# Import scipy library
import scipy as sp

# Import sklearn Libraries
import sklearn
import seaborn as sns

# Import module matlabplot for visaulizations
from matplotlib import pyplot as plt
from matplotlib import rcParams
import math

# Import Beautiful Soup library
import bs4
from BeautifulSoup import BeautifulSoup
import urllib2


### 2. List of Top 10 Fund Families based on largest Asset Under Management (AUM):

   This List includes Fund Families like Vanguard, American Funds, PIMCO, Fidelity Investments, Franklin Templeton Investments, BlacRock, T. Rowe Price, J.P.Morgan Funds, Oppenheimer Funds and Columbia. We will be fetching the data for the funds from the Morningstar website. Morningstar is one of the biggest source of information for almost all of the financial securities information. We will be using Web Scraping to fetch the Information for list of all fund families and it's funds. The fund's returns information and other important parameters for analysis.

In [256]:
# Defining a DataFrame for Fund Family which contains Fund Family and the Fund Family URL.
FFamily = pd.DataFrame(columns=['Fund_Family','MorningstarURL'])

# List of 10 largest Mutual Fund Families
FFamily.loc[0] = ['Vanguard','http://quicktake.morningstar.com/fundfamily/vanguard/0C00001YUF/fund-list.aspx']
FFamily.loc[1] = ['American Funds','http://quicktake.morningstar.com/fundfamily/american-funds/0C00001YPH/fund-list.aspx']
FFamily.loc[2] = ['PIMCO Funds','http://quicktake.morningstar.com/fundfamily/pimco/0C00004ALK/fund-list.aspx']
FFamily.loc[3] = ['T. Rowe Price','http://quicktake.morningstar.com/fundfamily/t-rowe-price/0C00001YZ8/fund-list.aspx']
FFamily.loc[4] = ['JP Morgan','http://quicktake.morningstar.com/fundfamily/jpmorgan/0C00001YRR/fund-list.aspx']
FFamily.loc[5] = ['Fidelity Investments','http://quicktake.morningstar.com/fundfamily/fidelity-investments/0C00001YR0/fund-list.aspx']
FFamily.loc[6] = ['Franklin Templeton Investments','http://quicktake.morningstar.com/fundfamily/franklin-templeton-investments/0C00004AKN/fund-list.aspx']
FFamily.loc[7] = ['BlackRock','http://quicktake.morningstar.com/fundfamily/blackrock/0C000034YC/fund-list.aspx']
FFamily.loc[8] = ['Columbia','http://quicktake.morningstar.com/fundfamily/columbia/0C00001YQG/fund-list.aspx']
FFamily.loc[9] = ['Oppenheimer Funds','http://quicktake.morningstar.com/fundfamily/oppenheimerfunds/0C00001YZF/fund-list.aspx']


In [257]:
# Display the Fund Family information
FFamily

Fund_Family  \
0                        Vanguard   
1                  American Funds   
2                     PIMCO Funds   
3                   T. Rowe Price   
4                       JP Morgan   
5            Fidelity Investments   
6  Franklin Templeton Investments   
7                       BlackRock   
8                        Columbia   
9               Oppenheimer Funds   

                                      MorningstarURL  
0  http://quicktake.morningstar.com/fundfamily/va...  
1  http://quicktake.morningstar.com/fundfamily/am...  
2  http://quicktake.morningstar.com/fundfamily/pi...  
3  http://quicktake.morningstar.com/fundfamily/t-...  
4  http://quicktake.morningstar.com/fundfamily/jp...  
5  http://quicktake.morningstar.com/fundfamily/fi...  
6  http://quicktake.morningstar.com/fundfamily/fr...  
7  http://quicktake.morningstar.com/fundfamily/bl...  
8  http://quicktake.morningstar.com/fundfamily/co...  
9  http://quicktake.morningstar.com/fundfamily/op...

### 3. Fund Family with Ticker:

   The below section contains code to fetch the list of Funds within each family. The web scraping from the Morningstar URL is done using Beautiful Soup. Ticker is a Unique idenitifer for a particular security similar to CUSIP and is a 5 character word(basically a combination of initials for the fund's name). However ticker makes more sense as it is more related to the Fund's family and name. 
   
   **E.g.** Fidelity funds have ticker starting with F whereas Vanguard funds have ticker starting with V.

In [259]:
# Fund Family DataFrame
Funds_family = pd.DataFrame(columns=['Fund_Name','Fund_Family','Fund_Ticker'])
i = 0

for index in range(0,len(FFamily)):
    # For each fund, fetch the MorningStar URL
    contenturl = FFamily.MorningstarURL[index]
    # Using urllib2 library.
    req = urllib2.Request(contenturl)
    page = urllib2.urlopen(req)
    # Using Beautiful Soup to read from page
    soup = BeautifulSoup(page)
    # Extract the information from the div which contains the class "syn_section_b1"
    table = soup.find("div", { "class" : "syn_section_b1" })
    # Loop to fetch all the fund tickers and its's names which is contained within href section of the URL
    for row in table.findAll('a'):
        # If we carefully observe the URL, the ticker information starts at 73
        if (row['href'][73:]) != '':
            Funds_family.loc[i] = [row.contents[0],FFamily['Fund_Family'][index],row['href'][73:]]
            i = i+1

In [260]:
# Fetched the Funds_Family head
Funds_family.head()

Fund_Name Fund_Family Fund_Ticker
0                  Vanguard 500 Index Inv    Vanguard       VFINX
1             Vanguard Balanced Index Inv    Vanguard       VBINX
2  Vanguard CA Interm-Term Tax-Exempt Inv    Vanguard       VCAIX
3    Vanguard CA Long-Term Tax-Exempt Inv    Vanguard       VCITX
4        Vanguard Capital Opportunity Inv    Vanguard       VHCOX

In [1503]:
# Total number of Funds loaded:

print " Total number of funds for analysis : ",len(Funds_family)

 Total number of funds for analysis :  1277


### 4. Load Benchmarks for each Fund :

   Each and every fund follows an index like the S&P 500 etc. which we call a benchmark, so that the fund's returns can be compared to the Index benchmark returns. Fetching the Fund's benchmark. 
   
   **Note**: Web Scraping usually takes time to run the code for these 1277 funds.

In [281]:
# Benchmark columns.
fund_benchmark_columns = ['Fund_Ticker','Benchmark_Index']
# Fund Benchmark DataFrame
fund_benchmark = pd.DataFrame(columns=fund_benchmark_columns)

# Loop for each fund for getting it's becnhmark
for i in range(0,len(Funds_family)):
    FUND_NAME = Funds_family['Fund_Ticker'][i]
    # try except exception block is used to ignore errors if any and moving forward for different funds.
    try:
        # Below is the AJAX request URL whose table contains the Benchmark
        ratingriskurl = "http://performance.morningstar.com/RatingRisk/fund/mpt-statistics.action?&t=XNAS:"+FUND_NAME+"&region=usa&culture=en-US&cur=&ops=clear&s=0P00001MK8&y=3&ep=true&comparisonRemove=null&benchmarkSecId=&benchmarktype="
        # Read the 0th value of the array
        mpt_statistics_bench = pd.read_html(ratingriskurl)[0]
        # Filtering all the not null values
        mpt_statistics_bench = mpt_statistics_bench[mpt_statistics_bench.Alpha.notnull()]
        # After filtering the row at index at 1 has the bechmark
        mpt_statistics_bench = mpt_statistics_bench.reset_index(drop=True).iloc[[1]]
        mpt_statistics_bench = mpt_statistics_bench.ix[:,0:2]
        mpt_statistics_bench.columns = fund_benchmark_columns
        # Append fund benchmark dataframe for each fund
        fund_benchmark = fund_benchmark.append(mpt_statistics_bench)
    except:
        # Those Funds which have error in finding benchmark are printed.
        print "Index : ",i,"No Benchmark Data Found in Morningstar for Fund : ",FUND_NAME

# Reset the fund_benchmark dataframe.
fund_benchmark = fund_benchmark.reset_index(drop=True)

Index :  173 No Benchmark Data Found in Morningstar for Fund :  PCPIX
Index :  227 No Benchmark Data Found in Morningstar for Fund :  PCIAX
Index :  228 No Benchmark Data Found in Morningstar for Fund :  POIAX
Index :  391 No Benchmark Data Found in Morningstar for Fund :  OAMAX
Index :  448 No Benchmark Data Found in Morningstar for Fund :  PEIAX
Index :  493 No Benchmark Data Found in Morningstar for Fund :  JSPAX


In [284]:
# Head of Fund_benchmark dataframe
fund_benchmark.head()

Fund_Ticker                   Benchmark_Index
0       VFINX                    S&P 500 TR USD
1       VBINX  Morningstar Moderate Target Risk
2       VCAIX         Barclays Municipal TR USD
3       VCITX         Barclays Municipal TR USD
4       VHCOX                    S&P 500 TR USD

### 5. Benchmarks and it's Morningstar Symbols:

   For fetching the returns information and other paramters information from Morningstar, we need to pass the benchmark code information alongwith the URI as a parameter. The Symbol is a unique for each benchmark.

In [1504]:
# Fetch all the distinct benchmarks for the above set of ~1277 funds
index_dup = fund_benchmark.drop_duplicates('Benchmark_Index')
# Total Distinct benchmarks for ~ 1277 funds
index_dup

Fund_Ticker                          Benchmark_Index
0         VFINX                           S&P 500 TR USD
1         VBINX         Morningstar Moderate Target Risk
2         VCAIX                Barclays Municipal TR USD
7         VTMGX                  MSCI ACWI Ex USA NR USD
10        VGOVX              Barclays US Agg Bond TR USD
13        VGENX                         MSCI ACWI NR USD
173       PCLAX       Morningstar Long-Only Commodity TR
184       PLMAX                BofAML USD LIBOR 3 Mon CM
252       PQTAX  Credit Suisse Mgd Futures Liquid TR USD

#### Storing the Mstar Symbol unique to each benchmark in a mstar_benchmark_symbol DataFrame.

In [258]:
mstar_benchmark_symbol = pd.DataFrame(index=range(0,len(index_dup)),columns=['Benchmark_Index','Mstar_Symbol'])

mstar_benchmark_symbol.loc[0] = ['S&P 500 TR USD','0P00001MK8']
mstar_benchmark_symbol.loc[1] = ['Morningstar Moderate Target Risk','0P0000J533']
mstar_benchmark_symbol.loc[2] = ['Barclays Municipal TR USD','0P00001G5X']
mstar_benchmark_symbol.loc[3] = ['MSCI ACWI Ex USA NR USD','0P00001MJB']
mstar_benchmark_symbol.loc[4] = ['Barclays US Agg Bond TR USD','0P00001G5L']
mstar_benchmark_symbol.loc[5] = ['MSCI ACWI NR USD','0P00001G8P']
mstar_benchmark_symbol.loc[6] = ['Morningstar Long-Only Commodity TR','0P00009FRD']
mstar_benchmark_symbol.loc[7] = ['BofAML USD LIBOR 3 Mon CM','0P00001L6O']
mstar_benchmark_symbol.loc[8] = ['Credit Suisse Mgd Futures Liquid TR USD','0P00001MK8']

mstar_benchmark_symbol

Benchmark_Index Mstar_Symbol
0                           S&P 500 TR USD   0P00001MK8
1         Morningstar Moderate Target Risk   0P0000J533
2                Barclays Municipal TR USD   0P00001G5X
3                  MSCI ACWI Ex USA NR USD   0P00001MJB
4              Barclays US Agg Bond TR USD   0P00001G5L
5                         MSCI ACWI NR USD   0P00001G8P
6       Morningstar Long-Only Commodity TR   0P00009FRD
7                BofAML USD LIBOR 3 Mon CM   0P00001L6O
8  Credit Suisse Mgd Futures Liquid TR USD   0P00001MK8

### 6. Fund DataFrame
    Fund DataFrame with all the above fetched information.

In [1505]:
# Merging the Funds_family dataframe and fund_benchmark dataframe.
fund_df = pd.merge(Funds_family,fund_benchmark,how='left',on='Fund_Ticker')
fund_df = fund_df[fund_df['Benchmark_Index'].notnull()].reset_index(drop=True)

# Printing the head for Fund DataFrame.
fund_df.head()

Fund_Name Fund_Family Fund_Ticker  \
0                  Vanguard 500 Index Inv    Vanguard       VFINX   
1             Vanguard Balanced Index Inv    Vanguard       VBINX   
2  Vanguard CA Interm-Term Tax-Exempt Inv    Vanguard       VCAIX   
3    Vanguard CA Long-Term Tax-Exempt Inv    Vanguard       VCITX   
4        Vanguard Capital Opportunity Inv    Vanguard       VHCOX   

                    Benchmark_Index  
0                    S&P 500 TR USD  
1  Morningstar Moderate Target Risk  
2         Barclays Municipal TR USD  
3         Barclays Municipal TR USD  
4                    S&P 500 TR USD

In [506]:
# Example for a fund with ticker = 'JMGIX'
fund_df[fund_df['Fund_Ticker'] == 'JMGIX']
#fund_df.shape

Fund_Name Fund_Family Fund_Ticker  \
441  JPMorgan Managed Income Institutional   JP Morgan       JMGIX   

                 Benchmark_Index  
441  Barclays US Agg Bond TR USD

    Exporting the Fund's DataFrame in a csv format. This file will be used to do analysis further for regression. We are storing the data as the above web scraping code take time to load data.

In [508]:
# Exporting the Fund's DataFrame in a csv format. 
fund_df.to_csv("Fund_Metadata.csv")

### 7. Returns Data
   Fetch 15 year Returns Data for each funds on an yearly basis. The fund's returns information is in an annualized format for each year. The Table contains the below information: So in all the fund contains 15 rows for each year till 2014.

Name | Meaning
:---: | :---: 
'Year' | Year 	
'FCNTX_Returns' | Fund's return for Year in percent
'S&P 500 TR USD' | Index returns (Fund's Benchmark returns) in percent
'Category (LB)' | Fund's Category Returns. For eg: Large Blend Category Returns in percent
'+/- S&P 500 TR USD' | + or - Fund's returns above/below Benchmark's Returns in percent
'+/- Category (LB)' | + or _ Fund's returns above/below Category's Returns in percent
'Annual_Net_Exp_Ratio' | Annual Net Expense Ratio for the Fund. Expenses to Investor to Fund Manager for managing the fund.
'Turnover_Ratio' | TurnOver Ratio for the Fund. For each year how much percentage of security allocations are changed. in percent
'Rank_In_Category' | Fund's Rank in the Category amongst the whole list of Category funds.

In [307]:
'''
This function "fnc_transpose" create transpose of fund_history data frame
and removes unwanted data from the data frame
It converts the data into float format.

Parameter - fund_history dataframe

Returns - transpose with required columns

'''
def fnc_transpose(fund_history):
    fund_history.index=fund_history[fund_history.columns[0]] # Make the parameters as the index
    fund_history = fund_history.drop(fund_history.columns[0], axis = 1)     
    fund_history = fund_history.transpose()    
    fund_history = fund_history.drop('Fund Category',1)    
    fund_history = fund_history.replace(to_replace=u'\u2014',value=0).astype(float) 
    fund_history = fund_history.ffill()
    fund_history = fund_history[(fund_history['Rank in Category']).notnull()] 
    return fund_history


In [308]:

'''
This function "get_performance_data" scraps performance data using Beautiful soup
and convert it to pandas dataframe.
It also handles missing values.

'''
def get_performance_data(url):
    soup = BeautifulSoup(urllib2.urlopen(url).read()) # read data using Beautiful Soup
    table = soup.find('table')                        # find the table
    rows = table.findAll('tr')                        # Find all tr rows with the table
    for i in range(len(rows)):                        # read table data for each table row
        if(i==0):
            header = rows[i].findAll('th')            # Assign first row as dataframe header
            column_names = [head.text for head in  header] 
            data = pd.DataFrame(columns=column_names)
        else:
            row = [0.0]*len(column_names)             # add table data to dataframe
            j=1
            row[0] = rows[i].find('th').text
            for cell in rows[i].findAll('td'):
                row[j] = cell.text
                j=j+1
            data.loc[i-1]=row
    # For those funds who donot have returns for 15 years contain --. Replacing those dashes with nan values.\
    data = data.replace(to_replace='&mdash;',value=np.nan)   #replace dash with null
    data = data.ffill()                                      # fill null values
    
    return data

   The Fund's returns information will be stored in a fund_returns object in a **key:value** format. The key will contains the Ticker information whereas the 15 year returns data alongwith other parameters for the Fund will the stored as a Value in a pandas DataFrame format. 
   An Empty DataFrame will be stored in the value when no returns information about the Fund is available.
   
   **Caution** : The below code takes more than **15 minutes** to scrap the whole data for ~1277 funds.
    

In [ ]:
# Initializing the Fund Returns object.
fund_returns = {}

# 15 year returns data for each fund.
for i in range(0,len(fund_df)):
    # Fetch the Ticker.
    fund_name = fund_df['Fund_Ticker'][i]
    # Fetch the Benchmark
    fund_benchmark_var = fund_df['Benchmark_Index'][i]
    # Key to be stored
    key = "fund_returns_"+fund_name
    # Fetching the Fund's benchmark and it s symbol
    symbol = (mstar_benchmark_symbol[mstar_benchmark_symbol['Benchmark_Index'] == fund_benchmark_var]['Mstar_Symbol']).values[0]
    # Fund Returns URL, passing the Fund Ticker, the bechmark symbol and 15 years of data to the URL
    fund_ret_url = "http://performance.morningstar.com/Performance/fund/performance-history-1.action?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur=&ops=clear&s="+symbol+"&ndec=2&ep=true&align=m&y=15&comparisonRemove=false&loccat=&taxadj=&benchmarkSecId=&benchmarktype="
    # Try Except block in case to encounter any errors
    try:
        # Calling get_performance_data function to scrape the data using Beautiful soup.
        fund_history = get_performance_data(fund_ret_url)
        # Calling fnc_transpose to Transpose the DataFrame
        fund_history = fnc_transpose(fund_history)
        fund_history = fund_history.reset_index()
        # Assigning the columns to the DataFrame.
        fund_history.columns = ['Year',fund_name+'_Returns',fund_benchmark_var,'Category (LB)','+/- S&P 500 TR USD','+/- Category (LB)','Annual_Net_Exp_Ratio','Turnover_Ratio','Rank_In_Category']
    except:
        # No returns Information then print and store an empty array in the value for that fund.
        print "No Returns Data available for Fund : ",fund_name
        # Creating an Empty DataFrame for fund with no returns
        fund_history = pd.DataFrame()
    # Assigning the DataFrame to the value
    fund_returns[key] = fund_history

Note that YTD indicates (Year to Date) meaning for this year (2014).

In [1506]:
fund_returns['fund_returns_FCNTX']

Year  FCNTX_Returns  S&P 500 TR USD  Category (LB)  +/- S&P 500 TR USD  \
0   1999          25.03           21.04          38.87                3.99   
1   2000          -6.80           -9.10         -13.05                2.30   
2   2001         -12.59          -11.89         -22.30               -0.70   
3   2002          -9.63          -22.10         -27.64               12.47   
4   2003          27.95           28.68          28.66               -0.73   
5   2004          15.07           10.88           7.81                4.19   
6   2005          16.23            4.91           6.71               11.32   
7   2006          11.54           15.79           7.05               -4.26   
8   2007          19.78            5.49          13.35               14.29   
9   2008         -37.16          -37.00         -40.67               -0.17   
10  2009          29.23           26.46          35.68                2.76   
11  2010          16.93           15.06          15.53                1.87   
12  2011          -0.14            2.11          -2.46               -2.25   
13  2012          16.26           16.00          15.34                0.25   
14  2013          34.15           32.39          33.92                1.76   
15   YTD          10.14           13.98          10.79               -3.84   

    +/- Category (LB)  Annual_Net_Exp_Ratio  Turnover_Ratio  Rank_In_Category  
0              -13.84                  0.62          177.00                21  
1                6.24                  0.84          166.00                40  
2                9.71                  0.91          141.00                51  
3               18.01                  0.99           80.00                 2  
4               -0.70                  0.98            0.98                39  
5                7.26                  0.92            0.92                 5  
6                9.52                  0.88           60.00                 3  
7                4.48                  0.89           76.00                14  
8                6.43                  0.89           56.00                19  
9                3.50                  0.94           78.00                21  
10              -6.46                  1.01           58.00                76  
11               1.40                  0.91           46.00                35  
12               2.32                  0.81           55.00                30  
13               0.92                  0.74           48.00                36  
14               0.23                  0.66           46.00                46  
15              -0.65                 -0.65           -0.65                61

Exporting the returns key:value information in a csv file. For each fund, a csv file is created. So for 1277 funds, individual returns information files are created. For FCNTX fund, FCNTX_Returns.csv will be the file name.

This is done in order to save time from fetching the returns information directly via web scraping. There were many issues most of the time with the Morningstar [www.morningstar.com](http://www.morningstar.com/) URL, as it was down most of the time and non responsive.

In [ ]:
# Export to csv files:

for i in range(0, len(fund_returns)):
    fund_key = fund_returns.keys()[i]
    # Fund Returns filename
    fund_filename = fund_returns.keys()[i][-5:]+"_RETURNS.csv"
    # Storing the Fund Returns values in a csv
    fund_returns[fund_key].replace(to_replace=u'\u2014',value='').to_csv(fund_filename)

### 8. Fund's MPT Statistics and Volatility Measures Data:

Here we will fetch the Fund's **MPT Statistics** like:

1. **R-Squared** :

    R-Squared is a statistical measure that represents the percentage of fund's movement that can be explained by movements in the benchmark index. A high R-squared (between 85 and 100) indicates the fund's performance patterns have been in line with the index. A fund with a low R-squared (70 or less) doesn't act much like the index.
    
2. **Beta** :
    
    Beta is a measure of volatility. A beta of 1 indicates that the security's price will move with the market. A beta of less than 1 means that the security will be less volatile than the market. A beta of greater than 1 indicates that the security's price will be more volatile than the market. 
    
3. **Alpha**: 

    A measure of performance on a risk-adjusted basis. Alpha takes the volatility (price risk) of a mutual fund and compares its risk-adjusted performance to a benchmark index. The excess return of the fund relative to the return of the benchmark index is a fund's alpha. A positive alpha of 1.0 means the fund has outperformed its benchmark index by 1%. Correspondingly, a similar negative alpha would indicate an underperformance of 1%.
        
4. **Treynor Ratio**:

    Treynor ratio is a risk-adjusted measure of return based on systematic risk. It is similar to the Sharpe ratio, with the difference being that the Treynor ratio uses beta as the measurement of volatility. Calculation : (Average Return of the Portfolio - Average Return of the Risk-Free Rate) / Beta of the Portfolio 


Fetch the **Volatilty Measures** like:

1. **Standard Deviation**

    A measure of the dispersion of a set of data from its mean. The more spread apart the data, the higher the deviation. Standard deviation is calculated as the square root of variance.

2. **Return**
    
    Returns information calculated based on Fund's NAV. Higher returns is what is the expectation.

3. **Sharpe Ratio**

     The Sharpe ratio is calculated by subtracting the risk-free rate - from the rate of return for a portfolio and dividing the result by the standard deviation of the portfolio returns. The Sharpe ratio tells us whether a portfolio's returns are due to smart investment decisions or a result of excess risk. The greater a portfolio's Sharpe ratio, the better its risk-adjusted performance has been. A negative Sharpe ratio indicates that a risk-less asset would perform better than the security being analyzed.
    
4. **Sortino Ratio**

    The Sortino ratio subtracts the risk-free rate of return from the portfolio’s return, and then divides that by the downside deviation. A large Sortino ratio indicates there is a low probability of a large loss.


Fetch the above Fund statistics for 3 years, 5 years , 10 years and 15 years. Perform analysis on this Data. This data will be used in Random Forest Classifier for performing predictive analytics.
The statistics information for each fund will be stored in an object in a key:value format. The statistics information for each fund will be DataFrame.

**Caution** : 
The below code takes approximately **1 hour 40 minutes** to load all the Fund statistics information for 1277 funds each separately for 3 year, 5 year, 10 year and 15 year. These parameters have to be passed to the Morningstar URI to get the associated information.

In [ ]:
# Year_Trailing array for years
Year_trailing = ['3','5','10','15']
# MPT Stats DataFrame columns
mpt_stats_columns = ['Fund_Ticker','Benchmark_Index', 'R_Squared','Beta','Alpha','Treynor_Ratio','Currency','Year_Trailing']
# Volatility Measures DataFrame columns
volatility_measures_columns = ['Fund_Ticker','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio','Bear_MktPercentile_Rank']

# Fund Statistics object
fund_statistics = {}

# Loop to hover for each fund
for yt in range(0,len(fund_df)):
    fund_name = fund_df['Fund_Ticker'][yt]
    fund_stats_benchmark = fund_df['Benchmark_Index'][yt]
    # Key value:
    key = "fund_statistics_"+fund_name
    # Fetch the Benchmark for each fund in loop
    symbol = (mstar_benchmark_symbol[mstar_benchmark_symbol['Benchmark_Index'] == fund_stats_benchmark]['Mstar_Symbol']).values[0]   
    # Initializing the mpt_statistics DataFrame for each fund. This DataFrame will be used to append statitics and measures information for different years.
    mpt_statistics = pd.DataFrame()
    # Another Loop for a fund and for 4 different years
    for i in Year_trailing:
        # MPT statistics URL passing fund ticker, year and morningstar benchmark ID
        mptstatsurl = "http://performance.morningstar.com/RatingRisk/fund/mpt-statistics.action?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur=&ops=clear&s="+fund_stats_benchmark+"&y="+i+"&ep=true&comparisonRemove=null&benchmarkSecId=&benchmarktype="
        # Volatility measures URL passing fund ticker, year and morningstar benchmark ID
        volmeasuresurl = "http://performance.morningstar.com/RatingRisk/fund/volatility-measurements.action?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur=&ops=clear&s="+fund_stats_benchmark+"&y="+i+"&ep=true&comparisonRemove=null&benchmarkSecId=&benchmarktype="
        try:
            # Read the MPT statistics URL information using pandas and store it in a DataFrame. Fetching the 0th value of the array.
            mpt_statistics_sub = pd.read_html(mptstatsurl)[0]
            # The read URL contains many nulls. Filtering the nan values
            mpt_statistics_sub = mpt_statistics_sub[mpt_statistics_sub.Alpha.notnull()]
            mpt_statistics_sub['Year_Trailing'] = i
            # Assigning columns to mpt_statistics_sub DataFrame
            mpt_statistics_sub.columns = mpt_stats_columns
            # Fetching the required information from the readed pandas table.
            mpt_statistics_sub = mpt_statistics_sub[(mpt_statistics_sub['Fund_Ticker'] == fund_name) & (mpt_statistics_sub['Benchmark_Index'] == fund_stats_benchmark)].reset_index(drop=True).tail(1)
            # Reading the volatility measures information using pandas and store it in a DataFrame. Fetching the 0th value of the array.
            volatility_measures = pd.read_html(volmeasuresurl)[0]
            # The read URL contains many nulls. Filtering the nan values
            volatility_measures = volatility_measures[volatility_measures.Return.notnull()].reset_index(drop = True)
            # Assigning columns to volatility_measures DataFrame
            volatility_measures.columns = volatility_measures_columns
            # Merging the mpt_statistics_sub and volatility_measures DataFrames on Fund_Ticker.
            mpt_statistics_sub = pd.merge(mpt_statistics_sub,volatility_measures,on='Fund_Ticker',how='inner')
            # Appending the merged information in the mpt_statistics information.
            mpt_statistics = mpt_statistics.append(mpt_statistics_sub)
        except:
            # Printing the Fund Information for which no data is available.
            print "Index : ",yt,", No Statitics available for Fund : ",fund_name," for ",i," years"
            # Assigning Empty DataFrame to store as value
            mpt_statistics = pd.DataFrame()
    # Assigning the value to the fund_statistics key for each fund in loop after reseting the index
    fund_statistics[key] = mpt_statistics.reset_index(drop=True)

In [354]:
# Fetching the Fund Statistics information for FCNTX fund.
fund_statistics["fund_statistics_FCNTX"]

Fund_Ticker Benchmark_Index R_Squared  Beta  Alpha Treynor_Ratio Currency  \
0       FCNTX  S&P 500 TR USD     83.49  1.00  -1.23         19.31      USD   
1       FCNTX  S&P 500 TR USD     83.49  1.00  -1.23         19.31      USD   
2       FCNTX  S&P 500 TR USD     83.49  1.00  -1.23         19.31      USD   
3       FCNTX  S&P 500 TR USD     83.49  1.00  -1.23         19.31      USD   

  Year_Trailing Std_Dev Return Sharpe_Ratio Sortino_Ratio  \
0             3    9.88  19.33         1.84          3.78   
1             5   12.57  15.61         1.22          2.22   
2            10   14.10  10.01         0.64          0.94   
3            15   13.62   7.56         0.46          0.66   

  Bear_MktPercentile_Rank  
0                       —  
1                   27.00  
2                       —  
3                       —

As we saw that the above code takes 1 hour 40 mins to scrape the data from Morningstar website. Hence storing the fund statistics and volaitilty measures data in csv file for each fund. The csv file names have FCNTX_STATS.csv file syntax.

In [349]:
# Export Data for Statistics in CSV files
for i in range(0, len(fund_statistics)):
    fund_key = fund_statistics.keys()[i]
    # Assigning file names for each fund
    fund_filename = fund_statistics.keys()[i][-5:]+"_STATS.csv"
    # Replacing the Unicode values
    fund_statistics[fund_key].replace(to_replace=u'\u2014',value='').to_csv(fund_filename)

For those funds which donot have 10 year or 15 year data is shown as --. These fund might be new funds in the market. For eg: VTMNX contains only 3 year and 5 year data.

In [355]:
fund_statistics.keys()[1]
fund_statistics["fund_statistics_VTWNX"]

Fund_Ticker                   Benchmark_Index R_Squared  Beta Alpha  \
0       VTWNX  Morningstar Moderate Target Risk     98.55     1   1.1   
1       VTWNX  Morningstar Moderate Target Risk     98.97  1.06  0.35   
2       VTWNX  Morningstar Moderate Target Risk         —     —     —   
3       VTWNX  Morningstar Moderate Target Risk         —     —     —   

  Treynor_Ratio Currency Year_Trailing Std_Dev Return Sharpe_Ratio  \
0         11.98      USD             3    6.41  12.01          1.8   
1          9.38      USD             5    9.03  10.06          1.1   
2             —      USD            10       —      —            —   
3             —      USD            15       —      —            —   

  Sortino_Ratio Bear_MktPercentile_Rank  
0          3.29                       —  
1          1.91                   18.00  
2             —                       —  
3             —                       —

### Funds MPT Statistics DataFrame for 3 years:

Creating a dataframe containing all the funds for 3 years. The 3 year analysis will be done further.

In [619]:
# Fund Stats columns
fund_stats_cols = fund_statistics["fund_statistics_FCNTX"].columns
# DataFrame for 3 years:
Fund_param_3 = pd.DataFrame(columns = fund_stats_cols)

# Looping through the fund_statistics object for each fund and retrieving the 3 year information.
for i in range(0,len(fund_statistics)):
    # Fetching the Key for each fund
    fund_stat_keys = fund_statistics.keys()[i]
    # Fetching the 3 year returns data and appending it to the Fund_param_3 dataframe 
    Fund_param_3 = Fund_param_3.append(fund_statistics[fund_stat_keys][fund_statistics[fund_stat_keys]['Year_Trailing'] == '3'])

# Dropping the Bear Market Percentile Market Rank as most of the values are not available
Fund_param_3 = Fund_param_3.drop('Bear_MktPercentile_Rank',1)
# Resetting the Fund_param_3 DataFrame index
Fund_param_3 = Fund_param_3.reset_index(drop=True)

In [620]:
# It is found that ticker JPCIX has Treynor_Ratio as incorrect. Assigning the correct value
Fund_param_3['Treynor_Ratio'][102] = 1954.64
# Replacing the columns with - with a random number -10001 for later removing the values from the dataframe
Fund_param_3[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']] = Fund_param_3[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']].replace(to_replace=u'\u2014',value=-10001).astype(float)
# Filtering the funds who donot have information on the parmaters
Fund_param_3 = Fund_param_3[Fund_param_3['R_Squared'] != -10001.0]
Fund_param_3 = Fund_param_3[Fund_param_3['Std_Dev'] != -10001.0].reset_index(drop=True)
# Print the length of the funds
print " Total # of funds with 3 year analysis data : ",len(Fund_param_3)

 Total # of funds with 3 year analysis data :  1122


In [621]:
# Print the header
Fund_param_3.head()

Fund_Ticker                   Benchmark_Index  R_Squared  Beta  Alpha  \
0       FRIFX                  MSCI ACWI NR USD      32.40  0.28   7.74   
1       VTWNX  Morningstar Moderate Target Risk      98.55  1.00   1.10   
2       JNBAX  Morningstar Moderate Target Risk      86.54  0.92   0.93   
3       FIOFX  Morningstar Moderate Target Risk      97.12  1.29  -0.77   
4       PTTRX       Barclays US Agg Bond TR USD      83.35  1.24   1.28   

  Treynor_Ratio Currency Year_Trailing  Std_Dev  Return  Sharpe_Ratio  \
0         43.38      USD             3     5.25   12.39          2.25   
1         11.98      USD             3     6.41   12.01          1.80   
2         11.82      USD             3     6.30   10.92          1.67   
3         10.17      USD             3     8.35   13.18          1.52   
4          4.00      USD             3     3.68    5.03          1.34   

   Sortino_Ratio  
0           4.31  
1           3.29  
2           3.00  
3           2.60  
4           2.18

In [639]:
# Exporting 3 years data to CSV file under ../Fund_Data/Fund_Stats_Annualized_data/Fund_statistics_3years.csv 

Fund_param_3.to_csv("Fund_statistics_3years.csv")

### Funds MPT Statistics DataFrame for 5 years:

Creating a dataframe containing all the funds for 5 years. The 5 year analysis will be done further.

In [622]:
# DataFrame for 5 years:
Fund_param_5 = pd.DataFrame(columns = fund_stats_cols)

# Looping through the fund_statistics object for each fund and retrieving the 5 year information.
for i in range(0,len(fund_statistics)):
    # Fetching the Key for each fund
    fund_stat_keys = fund_statistics.keys()[i]
    # Fetching the 5 year returns data and appending it to the Fund_param_5 dataframe 
    Fund_param_5 = Fund_param_5.append(fund_statistics[fund_stat_keys][fund_statistics[fund_stat_keys]['Year_Trailing'] == '5'])

# Dropping the Bear Market Percentile Market Rank as most of the values are not available
Fund_param_5 = Fund_param_5.drop('Bear_MktPercentile_Rank',1)
Fund_param_5 = Fund_param_5.reset_index(drop=True)

In [623]:
# Replacing the columns with - with a random number -10001 for later removing the values from the dataframe
Fund_param_5[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']] = Fund_param_5[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']].replace(to_replace=u'\u2014',value=-10001).astype(float)
# Filtering the funds who donot have information on the parmaters
Fund_param_5 = Fund_param_5[Fund_param_5['R_Squared'] != -10001.0]
Fund_param_5 = Fund_param_5[Fund_param_5['Std_Dev'] != -10001.0].reset_index(drop=True)
# Print the length of the funds
print " Total # of funds with 5 year analysis data : ",len(Fund_param_5)

 Total # of funds with 5 year analysis data :  1005


In [632]:
# Print the header
Fund_param_5.head()

Fund_Ticker                   Benchmark_Index  R_Squared  Beta  Alpha  \
0       FRIFX                  MSCI ACWI NR USD      55.22  0.30   8.48   
1       VTWNX  Morningstar Moderate Target Risk      98.97  1.06   0.35   
2       JNBAX  Morningstar Moderate Target Risk      88.95  0.96   0.44   
3       FIOFX  Morningstar Moderate Target Risk      98.47  1.37  -1.67   
4       PTTRX       Barclays US Agg Bond TR USD      59.01  0.98   1.02   

  Treynor_Ratio Currency Year_Trailing  Std_Dev  Return  Sharpe_Ratio  \
0         40.52      USD             5     5.84   12.18          1.99   
1          9.38      USD             5     9.03   10.06          1.10   
2          9.48      USD             5     8.62    9.20          1.06   
3          7.67      USD             5    11.65   10.58          0.92   
4          5.08      USD             5     3.60    5.06          1.37   

   Sortino_Ratio  
0           3.85  
1           1.91  
2           1.74  
3           1.51  
4           2.32

In [638]:
# Exporting the 5 years data to csv file
Fund_param_5.to_csv("Fund_statistics_5years.csv")

### Funds MPT Statistics DataFrame for 10 years :

Creating a dataframe containing all the funds for 10 years. The 10 year analysis will be done further.

In [625]:
# DataFrame for 10 years:
Fund_param_10 = pd.DataFrame(columns = fund_stats_cols)

# Looping through the fund_statistics object for each fund and retrieving the 10 year information.
for i in range(0,len(fund_statistics)):
    # Fetching the Key for each fund
    fund_stat_keys = fund_statistics.keys()[i]
    # Fetching the 10 year returns data and appending it to the Fund_param_10 dataframe 
    Fund_param_10 = Fund_param_10.append(fund_statistics[fund_stat_keys][fund_statistics[fund_stat_keys]['Year_Trailing'] == '10'])

# Dropping the Bear Market Percentile Market Rank as most of the values are not available
Fund_param_10 = Fund_param_10.drop('Bear_MktPercentile_Rank',1)
Fund_param_10 = Fund_param_10.reset_index(drop=True)

The funds with -- values are not avialble. Hence dropping those funds from analysis.

In [626]:
# Replacing the columns with - with a random number -10001 for later removing the values from the dataframe
Fund_param_10[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']] = Fund_param_10[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']].replace(to_replace=u'\u2014',value=-10001).astype(float)
# Filtering the funds who donot have information on the parmaters
Fund_param_10 = Fund_param_10[Fund_param_10['R_Squared'] != -10001.0]
Fund_param_10 = Fund_param_10[Fund_param_10['Std_Dev'] != -10001.0].reset_index(drop=True)
# Print the length of the funds
print " Total # of funds with 10 year analysis data : ",len(Fund_param_10)

 Total # of funds with 10 year analysis data :  760


In [627]:
# Print the header
Fund_param_10.head()

Fund_Ticker              Benchmark_Index  R_Squared  Beta  Alpha  \
0       FRIFX             MSCI ACWI NR USD      56.99  0.45   2.81   
1       PTTRX  Barclays US Agg Bond TR USD      69.53  1.01   1.24   
2       VGPMX             MSCI ACWI NR USD      56.66  1.44  -4.60   
3       PRGTX             MSCI ACWI NR USD      78.52  1.12   6.63   
4       FOSFX      MSCI ACWI Ex USA NR USD      95.43  1.03  -0.62   

  Treynor_Ratio Currency Year_Trailing  Std_Dev  Return  Sharpe_Ratio  \
0         11.99      USD            10     9.78    6.92          0.58   
1          4.54      USD            10     3.92    6.11          1.13   
2         -0.51      USD            10    31.67    0.78          0.14   
3          11.1      USD            10    20.84   13.94          0.66   
4          3.66      USD            10    19.83    5.29          0.29   

   Sortino_Ratio  
0           0.78  
1           2.00  
2           0.20  
3           1.02  
4           0.40

In [637]:
# Exporting the 10 years data to csv file
Fund_param_10.to_csv("Fund_statistics_10years.csv")

### Funds MPT Statistics DataFrame for 15 years:

Creating a dataframe containing all the funds for 15 years. The 15 year analysis will be done further.

In [628]:
# DataFrame for 15 years:
Fund_param_15 = pd.DataFrame(columns = fund_stats_cols)

# Looping through the fund_statistics object for each fund and retrieving the 15 year information.
for i in range(0,len(fund_statistics)):
    # Fetching the Key for each fund
    fund_stat_keys = fund_statistics.keys()[i]
    # Fetching the 15 year returns data and appending it to the Fund_param_15 dataframe 
    Fund_param_15 = Fund_param_15.append(fund_statistics[fund_stat_keys][fund_statistics[fund_stat_keys]['Year_Trailing'] == '15'])

# Dropping the Bear Market Percentile Market Rank as most of the values are not available
Fund_param_15 = Fund_param_15.drop('Bear_MktPercentile_Rank',1)
Fund_param_15 = Fund_param_15.reset_index(drop=True)

In [629]:
# Replacing the columns with - with a random number -10001 for later removing the values from the dataframe
Fund_param_15[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']] = Fund_param_15[['R_Squared','Beta','Alpha','Std_Dev','Return','Sharpe_Ratio','Sortino_Ratio']].replace(to_replace=u'\u2014',value=-10001).astype(float)
# Filtering the funds who donot have information on the parmaters
Fund_param_15 = Fund_param_15[Fund_param_15['R_Squared'] != -10001.0]
Fund_param_15 = Fund_param_15[Fund_param_15['Std_Dev'] != -10001.0].reset_index(drop=True)
# Print the length of the funds
print " Total # of funds with 15 year analysis : ",len(Fund_param_15)

 Total # of funds with 15 year analysis :  614


In [630]:
# Print the header
Fund_param_15.head()

Fund_Ticker              Benchmark_Index  R_Squared  Beta  Alpha  \
0       PTTRX  Barclays US Agg Bond TR USD      78.23  1.02   1.02   
1       VGPMX             MSCI ACWI NR USD      40.85  1.17   6.15   
2       FOSFX      MSCI ACWI Ex USA NR USD      94.99  1.04  -1.08   
3       VMRGX               S&P 500 TR USD      83.25  1.12  -3.23   
4       FSTFX    Barclays Municipal TR USD      78.13  0.41   0.02   

  Treynor_Ratio Currency Year_Trailing  Std_Dev  Return  Sharpe_Ratio  \
0          4.73      USD            15     4.03    6.80          1.17   
1          4.64      USD            15    30.21    7.42          0.33   
2          0.96      USD            15    19.34    2.97          0.15   
3         17.46      USD            15    17.56    4.90          0.25   
4          3.56      USD            15     2.03    3.43          0.70   

   Sortino_Ratio  
0           2.00  
1           0.47  
2           0.20  
3           0.35  
4           1.14

In [636]:
# Exporting the 15 years data to csv file

Fund_param_15.to_csv("Fund_statistics_15years.csv")



### 10. Fetching other detailed parameters:

Other parameters like TTM_Yield,Load, Portfolio Market Value, Total Assets ,Expense Ratio, FeeLevel, TurnOver Ratio, Category Information, Management Information and Returns information are equally important for Anaysis.

1) **TTM Yield** :

    A Company's Trailing Twelve Months(TTM) is a representation of its financial performance for a 12-month period, but typically not at its fiscal year end. Since quarterly reports rarely report how the company has done in the past 12 months, TTM tends to be calculated manually or found on various websites. Trailing 12 months figures can be calculated by subtracting the previous year's results from the same quarter as the most recent quarter reported and adding the difference to the latest fiscal year end results.
    
    
2) **Load**:

    Load fees are fees incurred by the investor for investing in the mutual fund. There is an entry load fee for purchasing some units within fund and exit load fees for exiting the fund within say 1 year.
    
    
3) **Portfolio Market Value**:

    The expected return of a market portfolio is identical to the expected return of the market as a whole. A bundle of investments that includes every type of asset available in the world financial market, with each asset weighted in proportion to its total presence in the market.
    
    
4) **Total Assets** :

    The sum of current and long-term assets owned by a mutual fund.
    
    
5) **Expense Ratio** : 

    Fees charged on the Investor per year for investing in a fund.
    
    
6) **FeeLevel** :
    
    The fees if it falls in the Low,Medium or High level bracket.
    
    
7) **TurnOver Ratio** :

    The percentage of a mutual fund's holdings that have been turned over or replaced with other holdings in a given year.
    
    
8) **Category Information** :

    The Category in which the fund is invested in, whether it is Large Cap Growth funds, or Large cap Value funds or Medium or Small Cap funds.
    
    
9) **Management Information**:

    Who is the current manager of the Fund and for how many years the manager is managing the fund. Manager's tenure is very important as the returns are based on his/her style of investing and getting maximum returns.


Web scraping used to get the parameters information. Webscraping in this case was difficult as most of the data in the html was not in a proper format. Here the pandas read_html didnt work and had to switch to Beautiful Soup for scraping.

**Caution** : The below code takes **10 minutes** to execute.

In [ ]:
# Column values
class_values = ['Fund_Ticker','TTM_Yield','Load','PortfolioMktValue','TotalAssets','ExpenseRatio','FeeLevel',
                'Turnover','Status','MinInvestment','Yield','MorningstarCategory','InvestmentStyle']

# Pandas DataFrame fund_other_params
fund_other_params = pd.DataFrame()
fund_other = pd.DataFrame(columns=class_values)

# For Loop for fetch the parameters for each fund.
for fd in range(0,len(fund_df)):
    fund_name = fund_df['Fund_Ticker'][fd]
    print "Fund other paramters : ",fund_name
    try:
        # Fetching the parameters of the fund like Expense Ratio
        fund_oth_params_url = "http://quotes.morningstar.com/fund/c-standardized?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur="
        fund_oth_params = pd.read_html(fund_oth_params_url,match='Max Front Load')[0]
        fund_oth_params.columns = fund_oth_params.loc[0]
        fund_oth_params = pd.DataFrame(pd.Series(fund_oth_params.loc[1])).T.reset_index(drop=True)
        fund_oth_params['Fund_Ticker'] = fund_name       
        
        # Fetching the annualized returns Since Inception
        fund_oth_params_ret = pd.read_html(fund_oth_params_url,match='Standardized Return')[0]
        fund_oth_params_ret.columns = ['Inception','Return','Tax_Return','Unnamed']
        fund_oth_params_ret = fund_oth_params_ret[fund_oth_params_ret['Inception'].str.contains('Inception',na=False)]
        fund_oth_params_ret['Fund_Ticker'] = fund_name
        fund_oth_params_ret = fund_oth_params_ret[['Fund_Ticker','Inception','Return']].reset_index(drop=True)
        var_inc = fund_oth_params_ret['Inception']
        start = var_inc[0].find('(')+1
        end = var_inc[0].find(')', start)
        fund_oth_params_ret['Inception'] = var_inc[0][start:end]

        # Fetching Fund Manager Information
        fund_management_url = "http://quotes.morningstar.com/fund/c-management?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur="
        fund_management = pd.read_html(fund_management_url,match='Start')[0]
        fund_management = fund_management[fund_management[0].notnull()].tail(1).reset_index(drop=True)
        fund_management.columns = ['Fund_Manager','Manager_Start_Date']
        fund_management['Fund_Ticker'] = fund_name
        
        # Merging above 3 dataframes fund_oth_params,fund_oth_params_ret and fund_management and storing it in fund_other_params dataframe.
        fund_other_params_sub = pd.merge(fund_oth_params_ret,fund_oth_params,on='Fund_Ticker',how='inner')
        fund_other_params_sub = pd.merge(fund_other_params_sub,fund_management,on='Fund_Ticker',how='inner')
        fund_other_params = fund_other_params.append(fund_other_params_sub)
        
        # Fetching other parameters like TurnOver Ratio etc.
        url_param = urllib2.urlopen("http://quotes.morningstar.com/chart/fund/c-banner?&t=XNAS:"+fund_name+"&region=usa&culture=en-US&cur=") # Opens URLS
        htmlSource = url_param.read()
        param_soup = bs4.BeautifulSoup(htmlSource)
        param_table = param_soup.find('table', attrs={'class': 'gr_table_b1'})
        param_rows = param_table.findAll('td')
        fund_values = {}
        i=0
        # The below code is used for stripping the spaces and only getting the required value.
        for tr in param_rows:
            fund_values_sub = tr.findAll('span')
            if len(fund_values_sub) != 0:
                fund_values[i] = fund_values_sub[0]
                if (i == 2):
                    var_total_asset = fund_values_sub[2].contents[3].contents[0].splitlines()[1].strip()          
                i = i+1

        for j in range(0,len(fund_values)):
            try:
                fund_values[j] = fund_values[j].contents[1].contents[0].splitlines()[1].strip()
            except:
                try:
                    fund_values[j] = fund_values[j].contents[2].splitlines()[1].strip()
                except:
                    fund_values[j] = fund_values[j].contents[0].splitlines()[1].strip()

        class_series = pd.Series([fund_name,fund_values[0],fund_values[1],fund_values[2],var_total_asset,fund_values[3],\
                                   fund_values[4],fund_values[5],fund_values[6],fund_values[7],fund_values[8],fund_values[9],\
                                   fund_values[10]])

        col_df = pd.DataFrame()
        col_df = col_df.append(class_series,ignore_index=True)
        col_df.columns=class_values
        fund_other = fund_other.append(col_df)
    except:
        # Print all the funds who had issues in loading the parameters information
        print "Issues in finding paramters for Fund : ",fund_name

# Merging fund_other and fund_other_params.
fund_other = pd.merge(fund_other,fund_other_params,on='Fund_Ticker',how='inner')        

In [1346]:
# Fetching the fund_other parameter data.

fund_other[fund_other['Fund_Ticker'] == 'FCNTX']

Fund_Ticker TTM_Yield  Load PortfolioMktValue TotalAssets ExpenseRatio  \
675       FCNTX     0.12%  None          16.8 bil  111.1  bil        0.66%   

    FeeLevel Turnover Status MinInvestment      ...       \
675      Low      46%   Open             $      ...        

    30-day Yield Unsubsidized(as of 09/30/2014)  \
675                                         NaN   

    30-day Yield Unsubsidized(as of 10/31/2014)    Fund_Manager  \
675                                         NaN  William Danoff   

    Gross Expense Ratio %   Inception Manager_Start_Date Max Back Load %  \
675                  0.67  05/17/1967         1990-09-17               —   

    Max Front Load % Net Expense Ratio % Return  
675                —                0.67  12.52  

[1 rows x 34 columns]

In [1496]:
# New Dataframe fund_other_copy
fund_other_copy = fund_other.copy()
# Dropping the Minimum Investment column as it is not significant
fund_other_copy = fund_other_copy.drop('MinInvestment',1)
# Replacing the unicode character to Nan
fund_other_copy.replace(to_replace=u'\u2014',value=np.nan,inplace=True)
# Encoding thr Fund Manager column which contains unicode characters to UTF-8 so to export to csv is possible
fund_other_copy['Fund_Manager'] = fund_other_copy['Fund_Manager'].apply(lambda x: x.encode("UTF-8"))
# Stripping the trailing % character in the below 4 columns
fund_other_copy['TTM_Yield_in_percent'] = fund_other_copy['TTM_Yield'].str.strip('%')
fund_other_copy['ExpenseRatio_in_percent'] = fund_other_copy['ExpenseRatio'].str.strip('%')
fund_other_copy['Turnover_in_percent'] = fund_other_copy['Turnover'].str.strip('%')
fund_other_copy['Yield_in_percent'] = fund_other_copy['Yield'].str.strip('%')

# Converting the columns PortfolioMktValue and TotalAssets columns to millions
fund_other_copy['PortfolioMktValue_in_millions'] = fund_other_copy['PortfolioMktValue'].apply(lambda x: float(x.strip(' bil'))*1000.0 if x[-4:] == ' bil'  else  x.strip(' mil'))
fund_other_copy['TotalAssets_in_millions'] = fund_other_copy['TotalAssets'].apply(lambda x: float(x.strip(' bil'))*1000.0 if x[-4:] == ' bil'  else  x.strip(' mil'))

# Changing column name for Inception, it is Fund Inception
fund_other_copy['Fund_Inception'] = fund_other_copy['Inception']
# Dropping the changed percent columns 
fund_other_copy = fund_other_copy.drop(['TTM_Yield','ExpenseRatio','Turnover','Yield','PortfolioMktValue','TotalAssets','Inception'],1)


In [1497]:
# Print the Header
fund_other_copy.head()

Fund_Ticker  Load FeeLevel   Status           MorningstarCategory  \
0       VFINX  None      Low     Open                   Large Blend   
1       VBINX  None      Low     Open           Moderate Allocation   
2       VCAIX  None      Low     Open  Muni California Intermediate   
3       VCITX  None      Low     Open          Muni California Long   
4       VHCOX  None      Low  Limited                  Large Growth   

  InvestmentStyle  30-day SEC Yield 30-day SEC Yield(as of 06/30/2014)  \
0     Large Blend               NaN                                NaN   
1     Large Blend               NaN                                NaN   
2   High/Moderate               NaN                                NaN   
3    Mid/Moderate               NaN                                NaN   
4    Large Growth               NaN                                NaN   

  30-day SEC Yield(as of 09/30/2014) 30-day SEC Yield(as of 10/31/2014)  \
0                                NaN                                NaN   
1                                NaN                                NaN   
2                                NaN                                NaN   
3                                NaN                                NaN   
4                                NaN                                NaN   

  30-day SEC Yield(as of 11/28/2014) 30-day SEC Yield(as of 11/30/2014)  \
0                                NaN                                NaN   
1                                NaN                                NaN   
2                                NaN                                NaN   
3                                NaN                                NaN   
4                                NaN                                NaN   

  30-day SEC Yield(as of 12/05/2014)  30-day Yield Unsubsidized  \
0                               1.79                        NaN   
1                               1.66                        NaN   
2                               1.55                        NaN   
3                               2.12                        NaN   
4                               0.52                        NaN   

  30-day Yield Unsubsidized(as of 01/31/2014)  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   

  30-day Yield Unsubsidized(as of 06/30/2014)  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   

  30-day Yield Unsubsidized(as of 07/31/2014)  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   

  30-day Yield Unsubsidized(as of 09/30/2014)  \
0                                         NaN   
1                                         NaN   
2                                         NaN   
3                                         NaN   
4                                         NaN   

  30-day Yield Unsubsidized(as of 10/31/2014)      Fund_Manager  \
0                                         NaN   Michael H. Buek   
1                                         NaN    Paul M. Malloy   
2                                         NaN  Adam M. Ferguson   
3                                         NaN   James M. D'Arcy   
4                                         NaN  M. Mohsin Ansari   

  Gross Expense Ratio % Manager_Start_Date Max Back Load % Max Front Load %  \
0                  0.17         1991-12-31             NaN              NaN   
1                  0.24         2013-03-22             NaN              NaN   
2    

In [1458]:
#pd.set_option('display.max_columns', None)
#fund_other1

In [1456]:
#pd.set_option('display.max_rows',None)
#fund_other1['Fund_Manager'] = fund_other1['Fund_Manager'].map(lambda x: x[:-2])
#fund_other1['Fund_Manager']
#pd.reset_option('display.max_rows')

In [1501]:
# Exporting the Fund Other Parameter data to csv file
fund_other_copy[['Fund_Ticker','Status','TTM_Yield_in_percent','PortfolioMktValue_in_millions','TotalAssets_in_millions',
             'ExpenseRatio_in_percent','Net Expense Ratio %','Gross Expense Ratio %','Turnover_in_percent','Yield_in_percent','Status','FeeLevel',\
             'Fund_Manager','Manager_Start_Date','Fund_Inception','MorningstarCategory','InvestmentStyle',\
             'FeeLevel','Load','Max Front Load %','Max Back Load %','Return']].to_csv("Fund_Parameters.csv")


### 11. Holding information for each fund:

This contains the Holding information for each fund. The total number of securities which is currently held by the fund in store. Again Data Scraping is used to fetch this information.

In [987]:
url_fund_holding = "http://portfolios.morningstar.com/fund/holdingsExport?exportType=details&&t=XNAS:FUSEX&region=usa&culture=en-US&cur=&dataType=0&holnum="

fund_holding =pd.read_csv(url_fund_holding)
fund_holding

﻿"Holdings"  Weighting                   Type  \
0                              Apple Inc       3.52                 EQUITY   
1                Exxon Mobil Corporation       2.25                 EQUITY   
2                         Microsoft Corp       2.11                 EQUITY   
3                      Johnson & Johnson       1.66                 EQUITY   
4                    General Electric Co       1.41                 EQUITY   
5         Berkshire Hathaway Inc Class B       1.39                 EQUITY   
6                       Wells Fargo & Co       1.37                 EQUITY   
7                    Procter & Gamble Co       1.29                 EQUITY   
8                           Chevron Corp       1.24                 EQUITY   
9                    JPMorgan Chase & Co       1.24                 EQUITY   
10            Verizon Communications Inc       1.13                 EQUITY   
11            S&P 500 Index Future Dec14       1.12  EQUITY INDEX - FUTURE   
12                            Pfizer Inc       1.03                 EQUITY   
13                              AT&T Inc       0.98                 EQUITY   
14           Bank of America Corporation       0.98                 EQUITY   
15                   Gilead Sciences Inc       0.92                 EQUITY   
16                            Intel Corp       0.92                 EQUITY   
17                        Merck & Co Inc       0.91                 EQUITY   
18                          Coca-Cola Co       0.90                 EQUITY   
19                         Citigroup Inc       0.88                 EQUITY   
20                    Google Inc Class A       0.88                 EQUITY   
21                    Google Inc Class C       0.87                 EQUITY   
22                  Facebook Inc Class A       0.85                 EQUITY   
23  International Business Machines Corp       0.83                 EQUITY   
24                           PepsiCo Inc       0.79                 EQUITY   
25                        Walt Disney Co       0.79                 EQUITY   
26                  Comcast Corp Class A       0.78                 EQUITY   
27       Philip Morris International Inc       0.76                 EQUITY   
28                          Qualcomm Inc       0.72                 EQUITY   
29                        Home Depot Inc       0.72                 EQUITY   
..                                   ...        ...                    ...   
70                           Monsanto Co       0.33                 EQUITY   
71    Mondelez International Inc Class A       0.32                 EQUITY   
72                              eBay Inc       0.32                 EQUITY   
73                 Costco Wholesale Corp       0.32                 EQUITY   
74                              EMC Corp       0.32                 EQUITY   
75               Duke Energy Corporation       0.32                 EQUITY   
76                  Colgate-Palmolive Co       0.31                 EQUITY   
77                       Express Scripts       0.31                 EQUITY   
78                        Starbucks Corp       0.31                 EQUITY   
79                           Walgreen Co       0.31                 EQUITY   
80                          Allergan Inc       0.31                 EQUITY   
81                  Lowe's Companies Inc       0.31                 EQUITY   
82              Simon Property Group Inc       0.30                 EQUITY   
83                       Dow Chemical Co       0.30                 EQUITY   
84                         Ford Motor Co       0.30                 EQUITY   
85                        Morgan Stanley       0.29                 EQUITY   
86                 Texas Instruments Inc       0.29                 EQUITY   
87                     EOG Resources Inc       0.28                 EQUITY   
88                  Lockheed Martin Corp       0.28                 EQUITY   
89                 Accenture PLC Class A       0.28

In [8]:
url_q_top_sector ="http://quotes.morningstar.com/fund/c-topSector?&t=XNAS:FUSEX&region=usa&culture=en-US&cur="

q_top_sector = pd.read_html(url_q_top_sector, match="Fund", header=0)[0]
q_top_sector


Unnamed: 0                        Fund  3 Yr High  \
0                         NaN                         NaN        NaN   
1                  Technology                       18.02      18.02   
2          Financial Services                       14.95      15.33   
3                  Healthcare                       14.32      14.32   
4                 Industrials                       11.02      11.68   
5           Consumer Cyclical                       10.22      11.23   
6  Portfolio Date: 09/30/2014  Portfolio Date: 09/30/2014        NaN   
7                         NaN                         NaN        NaN   
8  Portfolio Date: 09/30/2014                         NaN        NaN   

   3 Yr Low  Cat Avg  Unnamed: 5  Unnamed: 6  Fund.1  Cat Avg.1  
0       NaN      NaN         NaN         NaN     NaN        NaN  
1     16.77    17.04         NaN         NaN     NaN        NaN  
2     14.41    15.89         NaN         NaN     NaN        NaN  
3     11.81    14.35         NaN         NaN     NaN        NaN  
4     11.02    11.66         NaN         NaN     NaN        NaN  
5     10.17    10.77         NaN         NaN     NaN        NaN  
6       NaN      NaN         NaN         NaN     NaN        NaN  
7       NaN      NaN         NaN         NaN     NaN        NaN  
8       NaN      NaN         NaN         NaN     NaN        NaN

In [9]:
url_q_perf ="http://quotes.morningstar.com/fund/c-performance?&t=XNAS:FUSEX&region=usa&culture=en-US&cur=&benchmarkSecId=&benchmarktype="
q_perf = pd.read_html(url_q_perf, match="Fund", header=0)[0]
q_perf

Unnamed: 0       YTD      1 Mo  \
0                                   Growth of 10,000  11395.00  10555.00   
1                                               Fund     13.95      5.55   
2                                 +/- S&P 500 TR USD     -0.09     -0.01   
3                                       +/- Category      2.71      0.29   
4                                      % Rank in Cat     16.00     38.00   
5                                  # of Funds in Cat   1591.00   1681.00   
6  * Annualized returns. Data as of 11/24/2014. C...       NaN       NaN   

       1 Yr  Unnamed: 4      3 Yr   *      5 Yr  *.1     10 Yr  *.2  
0  11692.00         NaN  18956.00 NaN  20710.00  NaN  21457.00  NaN  
1     16.92         NaN     23.76 NaN     15.67  NaN      7.93  NaN  
2     -0.11         NaN     -0.10 NaN     -0.09  NaN     -0.06  NaN  
3      2.77         NaN      1.59 NaN      1.47  NaN      0.58  NaN  
4     18.00         NaN     28.00 NaN     19.00  NaN     26.00  NaN  
5   1553.00         NaN   1330.00 NaN   1191.00  NaN    794.00  NaN  
6       NaN         NaN       NaN NaN       NaN  NaN       NaN  NaN